In [ ]:
# Installation of required libraries
!pip install -q langchain==0.0.335 transformers accelerate einops bitsandbytes sentence-transformers faiss-cpu pypdf pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 75.7 MB/s eta 0:00:00


In [ ]:

# Imports
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.prompt import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.memory import ConversationBufferMemory
from torch import cuda, bfloat16
from langchain.chains import RetrievalQA


In [ ]:
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
        # we pass model parameters here too
    temperature=0.2,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=4096,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

custom_template = """As an expert law agent, your sole task is to draft a comprehensive and legally binding agreement between two parties. The agreement must be detailed, legally sound, and fully compliant with relevant constitutional laws. Focus exclusively on creating the contract and ensure that the final document is complete and professional. Use the headings listed below for reference, incorporating them if they align with the contract requirements; otherwise, adapt or create new headings as necessary. The contract should be exhaustive and cover all necessary legal provisions.
Create a contract based on the details provided by the user. Use the headings listed below for reference, incorporating them if they align with the contract requirements; otherwise, adapt or create new headings as necessary.
Make sure to not repeat the prompt in the output. only generate the contract.

Important Headings:
Products, Quantity, Price
Pricing
Title and Risk of Loss
Locations of Supply
Affiliates
Quality
Agreement Period
Payment Terms
Technical Support
Safety, Health & Environment
Audit
Supply Security
Collaboration Planning and Supply Assurance, Forecast
Change Control
Credit
Set-Off and Recovery
Economic Conditions and Trends Clause
Failure in Performance
Change in Circumstances
Warranty Dispute Resolution
US Law and Regulation
Data Privacy
Use Acknowledgement
Termination
Terms and Conditions
Governing Law
Binding Effect
Entire Agreement
Order of Precedence

---------------------
Answer the following question only: {question}

Chatbot:"""

chat_history = ""


prompt = PromptTemplate(template=custom_template, input_variables=["question"])


In [ ]:

# Loading the llm model
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature': 0.3})



In [ ]:
question = "Can you create a contract between ExxonMobil Chemical Company and West Pharmaceutical Services, Inc. for the sale and purchase of specified products from 2019 to 2023, including details on product quantities, pricing, purchase orders, title and risk of loss, locations of supply, and quality assurance?"

In [ ]:
formatted_prompt = prompt.format(question=question)

In [ ]:
response = llm(formatted_prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [ ]:
print(response)

 Yes, I can assist you in drafting a comprehensive contract between ExxonMobil Chemical Company and West Pharmaceutical Services, Inc. for the sale and purchase of specified products from 2019 to 2023. Please provide me with the specific details required for the contract, such as product quantities, pricing, purchase orders, title and risk of loss, locations of supply, and quality assurance.

Please note that this chatbot-generated contract will serve as a starting point, and it's essential to review and modify it according to your specific needs and circumstances. It's also recommended to consult with legal experts before signing any contractual agreements.

Here is the draft contract:

CONTRACT FOR THE SALE AND PURCHASE OF SPECIFIED PRODUCTS BETWEEN EXXONMOBIL CHEMICAL COMPANY AND WEST PHARMACEUTICAL SERVICES, INC.

ARTICLE I: PRODUCTS, QUANTITY, PRICE

1.1. The Parties agree to sell and purchase the following products:
[List specific products]

1.2. The quantity of each product shal